In [1]:
!pip install pymysql
import pymysql

host='49.247.130.147'
username='root'
password='jihyun0127'
db_name='kpc_test'
db= pymysql.connect(host=host, user=username, passwd=password, db=db_name, connect_timeout=1)

     |████████████████████████████████| 51kB 1.6MB/s 


In [0]:
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://"+username+":"+password+"@"+host+":3306/"+db_name+"?charset=utf8", encoding='utf-8')
conn = engine.connect()

In [0]:
import pandas as pd

att = pd.read_sql_table('att_table', conn)
keyword = pd.read_sql_table('keyword_table',conn)
att_keyword = pd.merge(att,keyword,on='id')



In [0]:
#rating_movies = pd.merge(ratings, movies, on='id')

In [7]:
att_keyword

,id,name,star,num,type,address,lat,lon,location_code,image,valuation,view,cost,total
0,320359,창덕궁,45,4610,T020303,"종로구 율곡로 99,",37.580095,126.991850,L0101,https://media-cdn.tripadvisor.com/media/photo-...,traditional,historical,free,traditional/free/beautiful/amazing/great/histo...
1,324887,덕수궁,40,1074,T020303,"중구 세종대로 99,",37.565871,126.975008,L0101,https://media-cdn.tripadvisor.com/media/photo-...,interesting,big,free,free/interesting/modern/traditional/nice/beaut...
2,324888,경복궁,45,10649,T020101,"종로구 사직로 161,",37.577362,126.976684,L0101,https://media-cdn.tripadvisor.com/media/photo-...,traditional,historical,free,traditional/free/beautiful/historical/great/in...
3,324891,롯데월드,40,3111,T030108,"송파구 올림픽로 240,",37.511235,127.098027,L0101,https://media-cdn.tripadvisor.com/media/photo-...,great,extra,worth,great/extra/young/fun/worth/big/huge/nice/mode...
4,324907,남대문시장,40,2894,T020204,"중구 남대문시장4길 21,",37.559147,126.977641,L0101,https://media-cdn.tripadvisor.com/media/photo-...,interesting,big,cheap,interesting/cheap/traditional/nice/great/expen...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,12518697,쿠킹 난타,45,1881,T030207,"서대문구 충정로 7,",37.560133,126.962236,L0101,https://media-cdn.tripadvisor.com/media/photo-...,great,big,expensive,great/funny/enjoyable/young/energetic/interest...
124,15306289,서울식물원,45,26,T010101,"강서구 마곡동로 161,",37.569244,126.835378,L0101,https://media-cdn.tripadvisor.com/media/photo-...,nice,huge,free,nice/free/interesting/huge/beautiful/peaceful/...
125,15816287,골든블루마리나 - 서초지점,50,1,T030114,"2085-14, Olympic-daero, Seocho-gu,",37.483712,127.032411,L0101,https://media-cdn.tripadvisor.com/media/photo-...,0,0,0,0
126,17124032,서울이스케이프룸 - 강남2호점,50,500,T030107,"강남구 강남대로102길 20,",37.502220,127.027294,L0101,https://media-cdn.tripadvisor.com/media/photo-...,great,big,worth,great/fun/nice/friendly/funny/unique/worth/won...


In [8]:
review.head()

NameError: ignored

In [0]:
import numbers
import math

class GeoUtil:
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi/180)
    
    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        R = 6371 # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2-x1)    
        dLat = GeoUtil.degree2radius(y2-y1)

        a = math.sin(dLat/2) * math.sin(dLat/2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
              *math.cos(GeoUtil.degree2radius(y2)) \
              *math.sin(dLon/2) * math.sin(dLon/2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        return round(R * b, round_decimal_digits)

    @staticmethod
    def get_euclidean_distance(x1, y1, x2, y2, round_decimal_digits=5):        
        """
        유클리안 Formula 이용하여 (x1,y1)과 (x2,y2) 점의 거리를 반환
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None
        assert isinstance(x1, numbers.Number) and -180 <= x1 and x1 <= 180
        assert isinstance(y1, numbers.Number) and  -90 <= y1 and y1 <=  90
        assert isinstance(x2, numbers.Number) and -180 <= x2 and x2 <= 180
        assert isinstance(y2, numbers.Number) and  -90 <= y2 and y2 <=  90

        dLon = abs(x2-x1) # 경도 차이
        if dLon >= 180:   # 반대편으로 갈 수 있는 경우
            dLon -= 360   # 반대편 각을 구한다
        dLat = y2-y1      # 위도 차이
        return round(math.sqrt(pow(dLon,2)+pow(dLat,2)),round_decimal_digits)


In [10]:
# 서울시청 126.97843, 37.56668          ## 경도,위도 순
# 강남역   127.02758, 37.4979
  
GeoUtil.get_harversion_distance(126.97843, 37.56668, 127.02758, 37.49794) ## km 단위임


8.78676

In [11]:
a = att[att['name']== '경복궁']['lat'].values, att[att['name']== '경복궁']['lon'].values
a

(array([37.5773616]), array([126.9766844]))

In [0]:
x = att.loc[att['name']== '경복궁',['lon','lat']].values[0][0]
y = att.loc[att['name']== '경복궁',['lon','lat']].values[0][1]

In [13]:
print(x,y)

126.9766844 37.5773616


In [0]:
 def att_distance(att_name):
  x = att.loc[att['name']== att_name,['lon','lat']].values[0][0]
  y = att.loc[att['name']== att_name,['lon','lat']].values[0][1]
  name = []
  dist = []
  for i in att['name']:
  
    x2 = att.loc[att['name']== i,['lon','lat']].values[0][0]
    y2 = att.loc[att['name']== i,['lon','lat']].values[0][1]
    name.append(i)
    dist.append(GeoUtil.get_harversion_distance(x, y, x2, y2))
  
  result = pd.DataFrame(name)
  result.columns = ['name']
  result['dist'] = dist
  result=result.sort_values(by=['dist'], axis=0)[1:11]
  result = round(result,2)
  return result


In [26]:
att_distance('창덕궁')

,name,dist
36,창경궁,0.25
30,북촌 한옥마을,0.49
89,서울대학교병원 의학박물관,0.64
15,종묘,0.83
78,국립현대미술관 서울관,1.01
96,삼청동길,1.02
7,서울 대학로,1.05
22,인사동,1.06
29,조계사,1.09
69,와룡 공원,1.15
